<img src = "https://drive.google.com/uc?export=view&id=1HYrQHu1AE2cEid3BpNxo5KK3tuZF1TCo" alt = "Encabezado MLDS" width = "100%">  </img>

# **Análisis de datos**
---

Este notebook es una plantilla que le puede servir como guía para el cuarto entregable del proyecto aplicado.

## **0. Integrantes del equipo de trabajo**
---

Por favor incluya los nombres completos y número de identificación de los integrantes del equipo de trabajo:

1. ---**INGRESE SU RESPUESTA**---
2. ---**INGRESE SU RESPUESTA**---
3. ---**INGRESE SU RESPUESTA**---

In [ ]:
# Instalamos los paquetes necesarios para el notebook en caso de trabajar desde Google Colab
!pip install wget pandas ydata-profiling ipywidgets

In [ ]:
import wget, pandas as pd

# Descargar el dataset transformado de la entrega anterior del repositorio publico
url = 'https://github.com/MLDS-UN-ProjectTeam/final-term-project/raw/main/resources/data/Saber_11_2023-2_-_Transformado.csv?download='
downloaded_file_name = 'Saber_11_2023-2_-_Transformado.csv'
wget.download(url = url, out = downloaded_file_name)

# Cargar en el notebook el conjunto de datos con las restricciones conocidas:
# - Delimitador: ¬ 
# - Quoting = 3 => Sin comillas para todas las variables
saber11_dataframe = pd.read_csv(downloaded_file_name, delimiter='¬', quoting=3)

In [ ]:
# Importamos matplotlib y scipy para las visualizaciones.
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import numpy as np
from scipy import stats
import scipy
import statsmodels.api as sm

## **1. Análisis y resultados**
---

Al adentrarse en el análisis de datos, es crucial adoptar un enfoque estratégico que responda a las preguntas de planteadas en el proyecto. La selección de variables para estudio requiere una evaluación cuidadosa de los objetivos específicos del proyecto, priorizando aquellas que se consideren fundamentales para obtener resultados significativos.

A lo largo de esta entrega, trate busque responder las siguientes preguntas

- ¿Cómo se aplicaron conceptos específicos de estadística descriptiva e inferencial en el análisis de datos y cuál fue el enfoque utilizado para interpretar los resultados de manera precisa?
- ¿Cómo se interpretaron los resultados de la estadística descriptiva y qué conclusiones se pueden extraer?
- ¿Cómo se relacionan los resultados del análisis de datos con los objetivos y preguntas planteadas inicialmente?
- ¿Qué implicaciones prácticas tienen los hallazgos y cómo podrían influir en la toma de decisiones?

In [ ]:
# Observamos la información básica del dataframe.
saber11_dataframe.info()

Se empezó por comprobar que la variable objetivo de estudio, "PUNT_GLOBAL", estuviera normalmente distribuída, puesto que es una pieza de información importante a la hora de evaluar su correlación con otras variables. Esto se hizo a través de un Gráfico Q-Q.

In [ ]:
plt.figure(dpi = 110) # Mejoramos la visibilidad de la gráfica.
stats.probplot(saber11_dataframe['PUNT_GLOBAL'], plot= plt)
plt.title('Gráfico Q-Q del Puntaje Global')
plt.show()

<img src = "./resources/images/punt_global_graf_qq.png" alt = "Gráfico Q-Q del Puntaje Global" width = "100%"></img>

A partir de la imagen es fácil observar que la mayoría de puntos azules, que representan a los datos, se encuentran sobre la línea roja, de donde se concluye que la variable de estudio SÍ está distribuída normalmente, con un leve comportamiento anormal en el cuantil más bajo.

### **1.1. Análisis de correlaciones**
---

- Analice la posible existencia de relaciones entre las variables, utilizando medidas de correlación o visualizaciones pertinentes.
- Detecte la presencia de redundancias entre las variables, con el objetivo de facilitar su posterior eliminación o selección en el análisis.
- ¿Cuáles fueron las variables seleccionadas para el análisis de correlaciones, y cómo se eligieron en función de los objetivos del proyecto?



Para el análisis de correlaciones se hará uso de una función que permite visualizar la relación entre las dos variables.

In [ ]:
# Función para evaluar correlaciones entre variables

def graficar_corr(df, var_a, var_b):

  r = df[var_a].corr(df[var_b]);
  fig, ax = plt.subplots(dpi = 110);
  df.plot.scatter(var_a, var_b, title=f"Coeficiente de correlación: {r :.2}", ax = ax,
                marker='o', linestyle='None', s = 70, alpha = 0.5, grid=True 
                );


**Variables a evaluar:**

* FAMI_ESTRATOVIVIENDA_N : Estrato socioeconómico de la vivienda.
*
* 
* 

La selección de estas variables estuvo fundamentada en la limpieza y evaluación hecha en entregas anteriores. Adicionalmente se consideró qué variables daban información más valiosa de acuerdo al objetivo principal del proyecto: encontrar qué factores socioeconómicos afectan de forma relevante y determinante el rendimiento de los estudiantes a la hora de presentar el exámen de estado.

Se empezó entonces por la variable más evidente, aquella que da cuenta del estrato socioeconómico de la vivienda del estudiante, así:

In [ ]:
graficar_corr(saber11_dataframe, 'FAMI_ESTRATOVIVIENDA_N', 'PUNT_GLOBAL')

### **1.2. Análisis de regresiones**
---

- Apartir de las correlaciones identificadas, ¿es posible llevar a cabo un análisis de regresiones para estimar el valor de alguna de las variables de interés?
- ¿Cuáles fueron las variables seleccionadas para el análisis de regresiones y cómo se eligieron en función de los objetivos del proyecto?


In [ ]:
# ---INGRESE SU CÓDIGO---

---**INGRESE SU RESPUESTA**---

### **1.3. Estadística inferencial (Opcional)**
---

- ¿Resulta pertinente calcular intervalos de confianza para estimar algún parámetro con confianza a partir de una muestra?
- ¿Es posible realizar pruebas de hipótesis para probar alguna teoría sobre un parámetro de la población?

In [ ]:
# ---INGRESE SU CÓDIGO---

---**INGRESE SU RESPUESTA**---

## **2. Resultados y visualizaciones**

### **2.1. Visualizaciones Estáticas**
---

* Incluya gráficos y visualizaciones estáticas que sean relevantes para los datos analizados.
* Explique cada visualización y cómo contribuye a la comprensión de los resultados.

In [ ]:
# ---INGRESE SU CÓDIGO---

### **2.2. Visualizaciones Interactivas**
---

* Cree visualizaciones interactivas, como gráficos dinámicos o tableros interactivos y proporcione instrucciones claras sobre cómo interactuar con ellos.
* Explore cómo estas visualizaciones ayudan en la exploración de datos y documente su análisis.

In [ ]:
# ---INGRESE SU CÓDIGO---

### **2.3. Visualización de Mapas Coropléticos (Opcional)**
---

* Si aplica mapas coropléticos para representar datos geoespaciales, incluya estas visualizaciones.
* Explique la elección de este tipo de visualización y cómo ésta contribuye a la comprensión de patrones geográficos y a la solución planteada.

In [ ]:
# ---INGRESE SU CÓDIGO---

### **2.4. Interpretación de Resultados**
---
* ¿Existen patrones o tendencias significativas en los datos?
* ¿Qué significan los resultados en términos prácticos?
* ¿Existen patrones o tendencias inesperadas en los datos que requieran una consideración especial o una investigación adicional?
* ¿Cuáles fueron los hallazgos principales que contribuirán al proceso de toma de decisiones?

Recuerde que la interpretación de resultados es una fase clave para extraer el significado de los datos y proporcionar respuestas o soluciones al problema inicial.

In [ ]:
# ---INGRESE SU CÓDIGO---

## **3. Conclusiones**
---
Asegúrese de incluir al menos una conclusión de cada unidad tratada en los *notebooks*. Estas conclusiones actúan como resúmenes breves que destacan los puntos clave y los aprendizajes esenciales de cada sección.


1.   
2.   
3.   
4.   



## **Créditos**
* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).
    
**Universidad Nacional de Colombia** - *Facultad de Ingeniería*